<a href="https://colab.research.google.com/github/LinaTobon/DataScienceFem/blob/master/FindSessionID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# importing libraries
import pandas as pd
import datetime

In [67]:
# reading files
master = pd.read_excel("/content/drive/MyDrive/TP/MASTER.xlsx",dtype = {'Call Start Time': str})
track = pd.read_excel("/content/drive/MyDrive/TP/SFTP_Track1.xlsx",dtype = {'Call Connect time': str})

In [68]:
master.head(2)

,Service,Phone,Session,Date,Call Start Time,Call End Time,Source Agent,Transfer Start Time,Transfer End Time,Transfer Target,Outcome
0,PCN Callback Manual,7.733926e+09,UB79BT61117DF8,2021-08-09,15:11:52,15:24:28,BELLIZZIPEDROZA.5,15:17:04,15:24:44,8555330360.0,AGENT - CUST RPC 7
1,PCN Callback Manual,7.733926e+09,UBD12T611180FF,2021-08-09,15:24:47,16:13:03,BELLIZZIPEDROZA.5,15:26:19,16:07:56,8665566001.0,NaN


In [69]:
track.head(2)

,Recording Name,Date,TPV Phone,LV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID
0,062221_8474777613_Giannini Bellizzi_Teleperfor...,2021-12-28,8.474778e+09,8.474778e+09,Giannini Bellizzi,0.0,12:52:08,NaT,NaN,2021-06-22 13:22:38,NaN
1,062321_3306991530_Carlos Velez_Teleperformance,2021-06-23,3.306992e+09,3.306992e+09,Carlos Velez,0.0,2021-06-23 17:24:33,NaT,NaN,2021-06-23 17:30:40,NaN


In [70]:
track.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recording Name     2996 non-null   object        
 1   Date               2996 non-null   datetime64[ns]
 2   TPV Phone          2996 non-null   float64       
 3   LV Phone           2486 non-null   float64       
 4   Agent              2996 non-null   object        
 5   Count              2996 non-null   float64       
 6   Call Connect time  2995 non-null   object        
 7   Other Date         10 non-null     datetime64[ns]
 8   Other LOB          16 non-null     object        
 9   TPV Time           1458 non-null   object        
 10  Session_ID         855 non-null    object        
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 257.6+ KB


In [71]:
master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13012 entries, 0 to 13011
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Service              13012 non-null  object        
 1   Phone                13012 non-null  float64       
 2   Session              13012 non-null  object        
 3   Date                 13012 non-null  datetime64[ns]
 4   Call Start Time      13012 non-null  object        
 5   Call End Time        12995 non-null  object        
 6   Source Agent         13012 non-null  object        
 7   Transfer Start Time  13012 non-null  object        
 8   Transfer End Time    13012 non-null  object        
 9   Transfer Target      13012 non-null  object        
 10  Outcome              11605 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 1.1+ MB


In [72]:
# tracks without sessionid
track_ns = track[track['Session_ID'].isna()]

In [73]:
len(track_ns)

2141

In [74]:
def validate_register(phone,time):
  "validates if phone and time columns are porperly formatted"
  status='valid'
  if (len(phone) != 10):
    status = 'not valid phone'
  else:
    try:
       datetime.datetime.strptime(time, '%H:%M:%S')
    except:
      status = 'not valid time'
  return  status

In [75]:
def cleaning_key(df,phone,date,time):
  "gives format to phone,date and time columns"

  df = df.copy(deep=True)

  # converting phone column to string
  df[phone] = df[phone].fillna(0).astype(int).astype(str)

  # Keeping rows that contain  LV PHONE
  df= df[df[phone]!='0']
  
  # cleaning call time
  df['Call_time_hms'] = df[time].str[-8:]
  df['Date_str'] = df['Date'].dt.strftime("%Y-%m-%d")
  return df

In [76]:
# getting date together
def call_date(date,time,status):
  "concatenates the date and time of call in a single column"
  Call_Date = None
  if status =='valid':
    Call_Date = pd.to_datetime(date+ time, \
                                      format='%Y-%m-%d%H:%M:%S')
  return Call_Date
  

In [77]:
# applying functions to track_ns file

track_ns = cleaning_key(track_ns,'LV Phone','Date','Call Connect time')
track_ns['status'] = track_ns.apply(lambda row : validate_register(row['LV Phone'], row['Call_time_hms']), axis = 1)
track_ns['Call_Date'] = track_ns.apply(lambda row : call_date(row['Date_str'], row['Call_time_hms'],row['status']), axis = 1)

In [78]:
# number of rows in track_ns file
len(track_ns)

1670

In [79]:
# applying functions to master file
master = cleaning_key(master,'Phone','Date','Call Start Time')
master['status'] = master.apply(lambda row : validate_register(row['Phone'], row['Call_time_hms']), axis = 1)
master['Call_Date'] = master.apply(lambda row : call_date(row['Date_str'], row['Call_time_hms'],row['status']), axis = 1)

In [80]:
master.drop_duplicates(subset=['Phone','Call_Date'], keep="first",inplace=True)

In [81]:
len(master)

9391

In [82]:
#checking if phone numbers match

unique_phone_master = pd.DataFrame(set(master['Phone']),columns= ['Phone'])
#unique_phone_track = track_ns.drop_duplicates(subset=['LV Phone'])
unique_phone_track = pd.DataFrame(set(track_ns['LV Phone']),columns= ['LV Phone'])
test1 = pd.merge(unique_phone_master,unique_phone_track,left_on=['Phone'],\
         right_on=['LV Phone'], how='right',indicator=True)

In [83]:
len(unique_phone_track)

1388

In [84]:
len(test1[test1['_merge']=='right_only'])

753

In [85]:
len(test1[test1['_merge']=='both'])

635

In [86]:
#test1[test1['_merge']=='right_only'].to_csv("NotFoundPhones.csv")

In [87]:
match_phone_track = test1[test1['_merge']=='both']

In [88]:
match_phone_track.drop(columns={'_merge'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


753 registros de numero de telefono en track no aparecen en la master y 635  si aparecen en ambas

we are going to match track rows that have an existing number in the master file

In [90]:
match_phone_track.drop(columns=['Phone'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [91]:
match_track = match_phone_track.merge(track_ns,left_on=['LV Phone'],\
         right_on=['LV Phone'], how='right',indicator=True)

match_track.rename(columns={'_merge':'match'},inplace=True)

In [92]:
match_track = match_track[match_track['match']=='both']

In [93]:
match_track.reset_index(drop=True,inplace=True)

In [94]:
len(match_track)

785

In [95]:
match_track.loc[(match_track['LV Phone']=='9089639817') & (match_track['Date']=='2022-01-19')]

,LV Phone,Recording Name,Date,TPV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID,Call_time_hms,Date_str,status,Call_Date,match
434,9089639817,011922_9089639817_Andres Lorduy_Teleperformance_1,2022-01-19,9.089640e+09,Andres Lorduy,1.0,14:54:00,NaT,NaN,NaN,NaN,14:54:00,2022-01-19,valid,2022-01-19 14:54:00,both
435,9089639817,011922_9089639817_Andres Lorduy_Teleperformance_2,2022-01-19,9.089640e+09,Andres Lorduy,2.0,14:54:00,NaT,NaN,NaN,NaN,14:54:00,2022-01-19,valid,2022-01-19 14:54:00,both


In [96]:
match_track.drop(columns={'match'},inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [97]:
master = master.loc[:,['Session','Phone','Date','Call Start Time','Call_Date','Date_str']]
master.reset_index(drop=True,inplace=True)

In [98]:
# joining two dataframes
master_track_ns = pd.merge(master,match_track,left_on=['Phone','Call_Date'],\
         right_on=['LV Phone','Call_Date'], how='right',indicator=True)

In [99]:
master_track_ns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 785 entries, 0 to 784
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Session            305 non-null    object        
 1   Phone              305 non-null    object        
 2   Date_x             305 non-null    datetime64[ns]
 3   Call Start Time    305 non-null    object        
 4   Call_Date          746 non-null    datetime64[ns]
 5   Date_str_x         305 non-null    object        
 6   LV Phone           785 non-null    object        
 7   Recording Name     785 non-null    object        
 8   Date_y             785 non-null    datetime64[ns]
 9   TPV Phone          785 non-null    float64       
 10  Agent              785 non-null    object        
 11  Count              785 non-null    float64       
 12  Call Connect time  785 non-null    object        
 13  Other Date         8 non-null      datetime64[ns]
 14  Other LOB 

In [136]:
master_track_ns[master_track_ns['_merge']=='both'].to_excel("/content/drive/MyDrive/TP/Matched_sessions2.xlsx")

In [101]:
RIP = master_track_ns.loc[(master_track_ns['_merge']=='right_only') & (master_track_ns['status']=='valid')]
RIP.rename(columns = {'_merge':'match'},inplace=True)
RIP.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [102]:
RIP.head()

,Session,Phone,Date_x,Call Start Time,Call_Date,Date_str_x,LV Phone,Recording Name,Date_y,TPV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID,Call_time_hms,Date_str_y,status,match
0,NaN,NaN,NaT,NaN,2021-11-19 16:34:26,NaN,3127855832,111921_3127855832_Andres Lorduy_Teleperformance,2021-11-19,3.127856e+09,Andres Lorduy,0.0,16:34:26,NaT,NaN,NaN,NaN,16:34:26,2021-11-19,valid,right_only
1,NaN,NaN,NaT,NaN,2021-11-22 15:01:56,NaN,2247950475,112221_2247950475_Chrison Jack_Teleperformance_1,2021-11-22,2.247950e+09,Chrison Jack,1.0,15:01:56,NaT,NaN,NaN,NaN,15:01:56,2021-11-22,valid,right_only
2,NaN,NaN,NaT,NaN,2021-11-22 15:15:03,NaN,2247950475,112221_2247950475_Chrison Jack_Teleperformance_2,2021-11-22,2.247950e+09,Chrison Jack,2.0,15:15:03,NaT,NaN,NaN,NaN,15:15:03,2021-11-22,valid,right_only
3,NaN,NaN,NaT,NaN,2021-11-22 18:29:29,NaN,3127218330,112221_3127218330_Jaime Mejia Ardila_Teleperfo...,2021-11-22,3.127218e+09,Jaime Mejia Ardila,2.0,18:29:29,NaT,NaN,NaN,NaN,18:29:29,2021-11-22,valid,right_only
4,NaN,NaN,NaT,NaN,2021-11-22 18:53:15,NaN,3127218330,112221_3127218330_Jaime Mejia Ardila_Teleperfo...,2021-11-22,3.127218e+09,Jaime Mejia Ardila,3.0,18:53:15,NaT,NaN,NaN,NaN,18:53:15,2021-11-22,valid,right_only


In [103]:
## FINDING SESSIONS

In [104]:
unique_session_master = master.drop_duplicates(subset=['Session'])

In [105]:
#RIP.sort_values(['Date_y','LV Phone',],inplace=True)
#iterable = RIP.groupby(['LV Phone','Date_y','Call_Date'],as_index= False).size()

In [106]:
#iterable

In [107]:
"""from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds
for index, row in iterable.iterrows():
  if index==0:
    #print(row['LV Phone'], row['Date_str_y'])
      phone = row['LV Phone']
      date = row['Date_str_y']
      subset_master = master.loc[(master['Phone']==phone) & (master['Date']==date)]
      subset_track  = RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]
      time_series  = subset_master['Call Start Time']
      
      for delta in range(1,5):
        time_change = datetime.timedelta(seconds=delta)
        Call_time_delta = subset_track['Call_time_hms'] - time_change
        Call_time_delta.dt.strftime("%H:%M:%S")
        try: Call_time_delta== time_series
        except:
          print(Call_time_delta)
          print(time_series)
          continue
        else:
          print(Call_time_delta)
          print(master.loc[(master['Phone']==phone) & (master['Date']==date)&(master['Call Connect time']==Call_time_delta)])
          #break
      RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]"""

'from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds\nfor index, row in iterable.iterrows():\n  if index==0:\n    #print(row[\'LV Phone\'], row[\'Date_str_y\'])\n      phone = row[\'LV Phone\']\n      date = row[\'Date_str_y\']\n      subset_master = master.loc[(master[\'Phone\']==phone) & (master[\'Date\']==date)]\n      subset_track  = RIP.loc[(RIP[\'LV Phone\']==phone) & (RIP[\'Date_y\']==date)]\n      time_series  = subset_master[\'Call Start Time\']\n      \n      for delta in range(1,5):\n        time_change = datetime.timedelta(seconds=delta)\n        Call_time_delta = subset_track[\'Call_time_hms\'] - time_change\n        Call_time_delta.dt.strftime("%H:%M:%S")\n        try: Call_time_delta== time_series\n        except:\n          print(Call_time_delta)\n          print(time_series)\n          continue\n        else:\n          print(Call_time_delta)\n          print(master.loc[(master[\'Phone\']==phone) & (master[\'Date\']==date)&(master[\'Call Connect time\']==Ca

In [108]:
"""from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds
import datetime as dt
for index, row in iterable.iterrows():
  if index<5:
    #print(row['LV Phone'], row['Date_str_y'])
      phone = row['LV Phone']
      date = row['Date_y']
      subset_master = unique_session_master.loc[(master['Phone']==phone) & (master['Date']==date)]
      subset_track  = RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]
      time_master  = subset_master['Call_Date']
      time_track = subset_track['Call_Date']
      diff = abs(time_master.item() - time_track.item()).total_seconds()
      
      #print(diff)
      try: diff < 15
      except:
        continue
      else:
        session = master['Session'].loc[(master['Phone']==phone) & (master['Date']==date)]
        RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date), 'Session']= session.values
        print(RIP['Session'].loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)])
          #break
          #RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]"""


SyntaxError: ignored

In [109]:
RIP.sort_values(['Date_y'],inplace=True)
iterable = RIP.groupby(['LV Phone','Date_y'],as_index= False).size()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [110]:
iterable

,LV Phone,Date_y,size
0,2018444872,2021-11-24,1
1,2149095615,2022-02-01,1
2,2178503238,2021-12-02,1
3,2178884306,2021-12-22,2
4,2195887963,2022-01-10,1
...,...,...,...
360,9014469425,2022-01-24,1
361,9084051903,2021-12-17,1
362,9089639817,2021-12-27,1
363,9089639817,2022-01-19,2


In [ ]:
"""from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds
import datetime as dt
for index, row in iterable.iterrows():
  if index<5:
    #print(row['LV Phone'], row['Date_str_y'])
      phone = row['LV Phone']
      date = row['Date_y']
      iteraciones = row['size']
      subset_master = unique_session_master.loc[(master['Phone']==phone) & (master['Date']==date)]
      subset_track  = RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]
      #time_master  = subset_master['Call_Date']
      #time_track = subset_track['Call_Date']
      
      for idndx1, row1 in subset_track.iterrows():
        for idndx2, row2 in subset_master.iterrows():
          diff = abs(row1['Call_Date'] - row2['Call_Date']).total_seconds()
          #print(phone + ' difference '+str(diff))
          if diff < 15:
            #print(diff)
            #print(diff<15)
            session = master['Session'].loc[(master['Phone']==phone) & (master['Date']==date)]
            RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date), 'Session']= session.values
            #print(RIP['Session'].loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)])
            break
          else:
            #print(diff)
            #print(diff<15)
            RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date), 'Session']= 'Not found'
            continue
          #RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]"""


In [125]:
from pandas._libs.tslibs.timedeltas import delta_to_nanoseconds
import datetime as dt
for index, row in iterable.iterrows():
  phone = row['LV Phone']
  date = row['Date_y']
  iteraciones = row['size']
  subset_master = unique_session_master.loc[(master['Phone']==phone) & (master['Date']==date)]
  subset_track  = RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]
  
  for indx1, row1 in subset_track.iterrows():
      for indx2, row2 in subset_master.iterrows():
        diff = abs(row1['Call_Date'] - row2['Call_Date']).total_seconds()
        #print(diff<15)
        #print(indx1)
        #print(indx2)
        if diff < 15:
          session = row2['Session']
          #print(session)
          RIP.loc[indx1, ['Session']] = session
          #print(RIP.loc[indx1, ['Session']])
          break
        else:
          #RIP.iloc[indx1]['Session'] = 'Not found'
          #RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date), 'Session']= 'Not found'
          continue

True
25
5307
U10FCFT619EC9D3
Session    U10FCFT619EC9D3
Name: 25, dtype: object
True
397
3643
U3894T61F9AB4E
Session    U3894T61F9AB4E
Name: 397, dtype: object
True
96
1322
U46ECT61A95AA1
Session    U46ECT61A95AA1
Name: 96, dtype: object
False
184
1836
False
184
1837
False
185
1836
False
185
1837
True
211
1924
U38EBT61DCAF50
Session    U38EBT61DCAF50
Name: 211, dtype: object
True
318
2830
U3620T61F1CB88
Session    U3620T61F1CB88
Name: 318, dtype: object
True
352
2944
U2362T61F2F1D0
Session    U2362T61F2F1D0
Name: 352, dtype: object
True
233
2145
U2493T61E866B9
Session    U2493T61E866B9
Name: 233, dtype: object
False
267
2286
True
267
2321
U1450T61EAE80B
Session    U1450T61EAE80B
Name: 267, dtype: object
True
353
3032
U839T61F40652
Session    U839T61F40652
Name: 353, dtype: object
True
168
1793
U2B01T61BB9B53
Session    U2B01T61BB9B53
Name: 168, dtype: object
False
340
2846
True
340
2847
U4F6DDT61F2EC29
Session    U4F6DDT61F2EC29
Name: 340, dtype: object
True
339
2846
U4F3D7T61F2EA93
Se

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


True
301
2615
U1E9ET61F0433E
Session    U1E9ET61F0433E
Name: 301, dtype: object
False
234
2083
True
234
2175
U38E0T61E89486
Session    U38E0T61E89486
Name: 234, dtype: object
True
52
5494
UFB6T61A6481C
Session    UFB6T61A6481C
Name: 52, dtype: object
True
355
3018
U6B6F5T61F463DF
Session    U6B6F5T61F463DF
Name: 355, dtype: object
True
255
2239
U2126T61E9B2FF
Session    U2126T61E9B2FF
Name: 255, dtype: object
False
281
2384
True
281
2420
UDC7T61EECC6A
Session    UDC7T61EECC6A
Name: 281, dtype: object
True
145
1546
U45B8AT61B22F0C
Session    U45B8AT61B22F0C
Name: 145, dtype: object
False
53
5487
False
53
5488
True
53
5569
U3A26T61A69A9E
Session    U3A26T61A69A9E
Name: 53, dtype: object
False
74
1129
True
74
1169
U2875T61A7C5FE
Session    U2875T61A7C5FE
Name: 74, dtype: object
True
370
3156
U30CCT61F45FB9
Session    U30CCT61F45FB9
Name: 370, dtype: object
True
2
5183
U915FT619BFA45
Session    U915FT619BFA45
Name: 2, dtype: object
False
1
5183
True
1
5225
U2E26T619BF733
Session    U2E26T6

In [126]:
RIP.loc[(RIP['LV Phone']=='2242501148') & (RIP['Date_y']=='2022-01-21')]

,Session,Phone,Date_x,Call Start Time,Call_Date,Date_str_x,LV Phone,Recording Name,Date_y,TPV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID,Call_time_hms,Date_str_y,status,match
267,U1450T61EAE80B,NaN,NaT,NaN,2022-01-21 12:06:20,NaN,2242501148,012122_2242501148_Luis Cadavid_Teleperformance,2022-01-21,2.242501e+09,Luis Cadavid,0.0,12:06:20,NaT,NaN,NaN,NaN,12:06:20,2022-01-21,valid,right_only


In [137]:
RIP.to_excel("/content/drive/MyDrive/TP/Matched_Sessions.xlsx")

In [130]:
#RIP.iloc[1]['Session']
RIP.loc[318]

Session                                                 U3620T61F1CB88
Phone                                                              NaN
Date_x                                                             NaT
Call Start Time                                                    NaN
Call_Date                                          2022-01-26 17:30:33
Date_str_x                                                         NaN
LV Phone                                                    2242011058
Recording Name       012622_2242011058_Jaime Mejia Ardila_Teleperfo...
Date_y                                             2022-01-26 00:00:00
TPV Phone                                                 2242011058.0
Agent                                               Jaime Mejia Ardila
Count                                                              0.0
Call Connect time                                             17:30:33
Other Date                                                         NaT
Other 

In [178]:
RIP['Session'].loc[(RIP['Phone']==phone) & (RIP['Date_y']==date)]

Series([], Name: Session, dtype: object)

In [150]:
RIP.loc[(RIP['LV Phone']==phone) & (RIP['Date_y']==date)]

,Session,Phone,Date_x,Call Start Time,Call_Date,Date_str_x,LV Phone,Recording Name,Date_y,TPV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID,Call_time_hms,Date_str_y,status,match
25,NaN,NaN,NaT,NaN,2021-11-24 18:25:09,NaN,2018444872,112421_2018444872_Diego Hoyos_Teleperformance,2021-11-24,2.018445e+09,Diego Hoyos,0.0,18:25:09,NaT,NaN,NaN,NaN,18:25:09,2021-11-24,valid,right_only


In [248]:
subset_track

,Session,Phone,Date_x,Call Start Time,Call_Date,Date_str_x,LV Phone,Recording Name,Date_y,TPV Phone,Agent,Count,Call Connect time,Other Date,Other LOB,TPV Time,Session_ID,Call_time_hms,Date_str_y,status,match,NSession
25,1,NaN,NaT,NaN,2021-11-24 18:25:09,NaN,2018444872,112421_2018444872_Diego Hoyos_Teleperformance,2021-11-24,2.018445e+09,Diego Hoyos,0.0,18:25:09,NaT,NaN,NaN,NaN,18:25:09,2021-11-24,valid,right_only,1
